# Pneumonia_Chest_X-Ray

CNN image detection of pneumonia from chest xrays.  
Project By: [Uzair Bin Asim](https://github.com/Uzair05)

## Download dataset

In [ ]:
#!pip install --upgrade kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [2]:
import os 
import zipfile

try:
    if not os.path.isdir("./data"):
        os.mkdir("./data")
except Exception as err:
    print(f"Error in creating data/:\t{err}")

if os.path.isfile("./chest-xray-pneumonia.zip"):
    localzip = "./chest-xray-pneumonia.zip"
    with zipfile.ZipFile(localzip, 'r') as zip_ref:
        zip_ref.extractall('data/')

In [ ]:
## Clean the dataspace
# !rm -rf data/chest_xray/__MACOSX
# !cd data/ && find -type f | grep -E "\.DS_Store" | xargs -I{} rm {}

## Handle Data
Create the train, test, and validation datasets.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
main_path = "data/chest_xray/chest_xray"

test_dir, train_dir = os.path.join(main_path, "test"), os.path.join(main_path, "train")
validation_dir = os.path.join(main_path, "val") 

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, 
    rotation_range=10, 
    width_shift_range=0.3, 
    height_shift_range=0.3, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
)
test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)
val_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)
val_generator = val_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)

## Create Model

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.optimizers import RMSprop

In [ ]:
pretrained_model = EfficientNetB3(
    include_top=False, 
    weights='imagenet', 
    input_shape=(150,150,3)
)

for layer in pretrained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pretrained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout

In [ ]:
x = Flatten()(last_output)
x = Dense(32, activation=tf.nn.relu)(x)
x = Dense(64, activation=tf.nn.relu)(x)
x = Dense(64, activation=tf.nn.relu)(x)
x = Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(pretrained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)


## Train Model - v1

In [ ]:
history = model.fit(
    train_generator, 
    validation_data = test_generator,
    #steps_per_epoch = 100,
    epochs = 20,
    validation_steps = 50,
    verbose = 2
)

## Evaluation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
#loss = history.history['loss']
#val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

## Save Model

In [ ]:
#tf.keras.Model.save_model()

## Train Model - v2

### Load appended data
Author provided extra data in a different directory during update.

In [ ]:
main_path = "data/chest_xray/"

test_dir, train_dir = os.path.join(main_path, "test"), os.path.join(main_path, "train")
validation_dir = os.path.join(main_path, "val") 

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, 
    rotation_range=10, 
    width_shift_range=0.3, 
    height_shift_range=0.3, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
)
test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)
val_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)
val_generator = val_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)

### Second Training

In [ ]:
history = model.fit(
    train_generator, 
    validation_data = test_generator,
    #steps_per_epoch = 100,
    epochs = 20,
    validation_steps = 50,
    verbose = 2
)

### Second Evaluation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
#loss = history.history['loss']
#val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

### Save Model
Overwrite Previous Save

In [ ]:
#tf.keras.Model.save_model()